In [1]:
using Revise

In [2]:
using LearningMPC
using LearningMPC.Models
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using RigidBodyDynamics.PDControl
using MomentumBasedControl
using MeshCatMechanisms
using RigidBodySim
using RigidBodySim.Visualization.MeshCatInterface
import LCPSim
using JLD2
using Blink

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='pkg/WebIO/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='pkg/WebIO/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [3]:
robot = BoxAtlas();

In [27]:
mvis = MechanismVisualizer(robot)
open(mvis, Window())

Blink.AtomShell.Window(3, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 2149`, ProcessRunning), TCPSocket(RawFD(53) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(3, WebSockets.WebSocket(7, TCPSocket(RawFD(58) active, 0 bytes waiting), CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 3, Nullable{Any}(true))))

In [5]:
@load "../2018-05-08-box-atlas-longer-miqp/boxatlas.jld2" net lqrsol mpc_params
mpc_params_data = mpc_params
mpc_params = MPCParams(robot)
mpc_params.Δt = mpc_params_data["Δt"]
mpc_params.horizon = mpc_params_data["horizon"]
net_cost = LearningMPC.LearnedCost(lqrsol, net)
net_mpc_params = MPCParams(robot)
net_mpc_params.horizon = 1
net_mpc_controller = MPCController(robot, net_mpc_params, net_cost, [lqrsol]);
full_mpc_controller = MPCController(robot, mpc_params, lqrsol, [lqrsol, net_mpc_controller]);
lqr_mpc_controller = MPCController(robot, (p = MPCParams(robot); p.horizon=1; p), lqrsol, [lqrsol])

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


(::MPCController) (generic function with 1 method)

In [6]:
rootframe = root_frame(mechanism(robot))
floor = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism(robot), floor)
wall = HalfSpace3D(Point3D(rootframe, 0., 1.0, 0.), FreeVector3D(rootframe, 0., -1., 0.))
add_environment_primitive!(mechanism(robot), wall)

feet = Dict(:left => findbody(mechanism(robot), "l_foot_sole"), :right => findbody(mechanism(robot), "r_foot_sole"))
hands = Dict(:left => findbody(mechanism(robot), "l_hand_mount"), :right => findbody(mechanism(robot), "r_hand_mount"));

contact_model = SoftContactModel(hunt_crossley_hertz(k = 500e3), ViscoelasticCoulombModel(0.8, 20e3, 100.))

for (side, body) in feet
    add_contact_point!(body, 
        ContactPoint(Point3D(default_frame(body), 0., 0, 0), 
                     contact_model))
end

for (side, body) in hands
    add_contact_point!(body, 
        ContactPoint(Point3D(default_frame(body), 0., 0, 0), 
                     contact_model))
end


In [8]:
momentum_based_controller = MomentumBasedController(mechanism(robot))
contacts = add_mechanism_contacts!(momentum_based_controller)

Dict{RigidBodyDynamics.RigidBody{Float64},Array{MomentumBasedControl.ContactSettings,1}} with 14 entries:
  world               => MomentumBasedControl.ContactSettings[]
  wall                => MomentumBasedControl.ContactSettings[]
  floating_base_dummy => MomentumBasedControl.ContactSettings[]
  root                => MomentumBasedControl.ContactSettings[]
  r_foot_sole         => MomentumBasedControl.ContactSettings[MomentumBasedCont…
  floor               => MomentumBasedControl.ContactSettings[]
  r_hand_mount_dummy  => MomentumBasedControl.ContactSettings[]
  l_foot_sole         => MomentumBasedControl.ContactSettings[MomentumBasedCont…
  pelvis              => MomentumBasedControl.ContactSettings[]
  r_foot_sole_dummy   => MomentumBasedControl.ContactSettings[]
  r_hand_mount        => MomentumBasedControl.ContactSettings[MomentumBasedCont…
  l_hand_mount_dummy  => MomentumBasedControl.ContactSettings[]
  l_hand_mount        => MomentumBasedControl.ContactSettings[MomentumBased

In [9]:
μ = 0.8
floor_normal = floor.outward_normal
wall_normal = wall.outward_normal

FreeVector3D in "world": [0.0, -1.0, 0.0]

In [44]:
# FIXME: referencing global variables
function lqr_high_level_control(τ::AbstractVector, t::Number, state::MechanismState)
    # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(momentum_based_controller)
    MomentumBasedControl.reset!(momentum_based_controller)
    regularize_joint_accels!(momentum_based_controller, 0.05)
    
    for (side, body) in feet
        for contactsettings in contacts[body]
            pt_in_world = transform(state, contactsettings.point, root_frame(state.mechanism))
            set!(contactsettings, 1e-4, μ, transform(state, floor_normal, default_frame(body)))
        end
    end
    
    momentum_based_controller.terminalstatetask.x .= lqrsol.x0
    momentum_based_controller.terminalstatetask.Q .= lqrsol.S
    momentum_based_controller.terminalstatetask.q .= 0
    
    # Low level control
    momentum_based_controller(τ, t, state)
    τ
end

lqr_high_level_control (generic function with 1 method)

In [49]:
# FIXME: referencing global variables
function net_high_level_control(τ::AbstractVector, t::Number, state::MechanismState)
    # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(momentum_based_controller)
    MomentumBasedControl.reset!(momentum_based_controller)
    regularize_joint_accels!(momentum_based_controller, 0.05)
    
    for (side, body) in feet
        for contactsettings in contacts[body]
            pt_in_world = transform(state, contactsettings.point, root_frame(state.mechanism))
            max_normal_force = pt_in_world.v[3] >= 1e-2 ? 0.0 : Inf
            set!(contactsettings, 1e-4, μ, 
                transform(state, floor_normal, default_frame(body)),
                max_normal_force)
        end
    end
    
    body = hands[:left]
    for contactsettings in contacts[body]
        pt_in_world = transform(state, contactsettings.point, root_frame(state.mechanism))
        max_normal_force = (pt_in_world.v[2] <= (1 - 1e-2)) ? 0.0 : Inf
            
        set!(contactsettings, 1e-1, μ, transform(state, wall_normal, default_frame(body)))
    end
    
    momentum_based_controller.terminalstatetask.x .= lqrsol.x0
    momentum_based_controller.terminalstatetask.Q .= lqrsol.Q
    q0, q = net_cost.tangent_net(vcat(configuration(state), velocity(state)))
    momentum_based_controller.terminalstatetask.q .= vec(Flux.Tracker.data(q))
#     momentum_based_controller.terminalstatetask.Q .= 0
#     momentum_based_controller.terminalstatetask.q .= 0
    
    # Low level control
    momentum_based_controller(τ, t, state)
    τ
end

net_high_level_control (generic function with 1 method)

In [50]:
effort_bounds = LCPSim.all_effort_bounds(mechanism(robot))

function effort_limiter(τ::AbstractVector, t::Number, state::MechanismState)
    τ .= clamp.(τ, effort_bounds)
    τ
end

position_bounds = LCPSim.all_configuration_bounds(mechanism(robot))

function bounds_enforcer(τ::AbstractVector, t::Number, state::MechanismState)
    # TODO: handle q̇ vs v correctly
    for i in 1:num_positions(state)
        kp = 1000
        kd = 0.1 * kp
        if configuration(state)[i] > position_bounds[i].upper
            violation = configuration(state)[i] - position_bounds[i].upper
            τ[i] -= kp * violation
            τ[i] -= kd * velocity(state)[i]
        elseif configuration(state)[i] < position_bounds[i].lower
            violation = position_bounds[i].lower - configuration(state)[i]
            τ[i] += kp * violation 
            τ[i] -= kd * velocity(state)[i]
        end
    end
    τ
end

function compose(controllers...)
    function (τ, t, state)
        for c in controllers
            c(τ, t, state)
        end
        τ
    end
end

    

compose (generic function with 1 method)

In [53]:
state = nominal_state(robot)
set_velocity!(state, findjoint(mechanism(robot), "floating_base"), [0.5, 0, 0.])
final_time = 5.
Δt = 0.01

composed = PeriodicController(similar(velocity(state)), Δt, compose(lqr_high_level_control, effort_limiter))
controlcallback = DiffEqCallbacks.PeriodicCallback(composed)
composed = compose(composed, bounds_enforcer)

dynamics = Dynamics(mechanism(robot), composed)
problem = ODEProblem(dynamics, state, (0., final_time), callback = CallbackSet(controlcallback, CallbackSet(mvis, state, max_fps = 30.)))

@time sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = Δt);

  5.492411 seconds (16.94 M allocations: 748.119 MiB, 12.32% gc time)


In [52]:
RigidBodySim.animate(mvis, state, sol)